# Instruction Fine-tuning sobre un LLM Base

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Enunciado del caso práctico</h2>

En este caso práctico, se propone al alumno la realización de instruction fine-tuning sobre el LLM [Flan-T5-small](https://huggingface.co/google/flan-t5-small) con el objetivo de que sea capaz de resumir artículos de periódico en el idioma Español.

</div>

# Resolución del caso práctico

## 0. Instalación de librerías externas

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install accelerate
!pip install datasets
!pip install evaluate
!pip install rouge_score

## 1. Comportamiento de [Flan-T5-small](https://huggingface.co/google/flan-t5-small) sin Fine-tuning

### Lectura del modelo y tokenizador

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Importamos el tokenizador
tokenizer_FT5 = T5Tokenizer.from_pretrained("google/flan-t5-small")

# Importamos el modelo pre-entrenado
model_FT5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

### Generación de texto

In [ ]:
text = """Astrónomos detectaron una misteriosa ráfaga de ondas de radio que tardó \
8.000 millones de años en llegar a la Tierra. La ráfaga rápida de radio es una de \
las más distantes y energéticas jamás observadas. Las ráfagas rápidas de radio \
(FRB, por sus siglas en inglés) son intensos estallidos de ondas de radio de \
milisegundos de duración cuyo origen se desconoce. La primera FRB se descubrió \
en 2007 y, desde entonces, se han detectado cientos de estos rápidos destellos \
cósmicos procedentes de puntos distantes de todo el universo."""

In [ ]:
text = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [ ]:
text = """"El telescopio Hubble, lanzado al espacio en 1990, ha proporcionado imágenes \
impresionantes del universo y ha ayudado a los científicos a comprender mejor la cosmología."""

In [ ]:
prompt_template = f"Resume el siguiente articulo:\n\n{text}"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5(prompt_template, return_tensors="pt").input_ids.to("cuda")

# Generamos los siguientes tokens
outputs = model_FT5.generate(prompt_tokens, max_length=200)

# Transformamos los tokens generados en texto
print(tokenizer_FT5.decode(outputs[0]))

## 2. Selección y preparación del conjunto de datos

El conjunto de datos que vamos a utilizar para la realización del fine-tuning se corresponde con MLSUM.

MLSUM es el primer conjunto de datos de resumen multilingüe a gran escala. Obtenido de periódicos en línea, contiene más de 1.5 millones de pares de artículo/resumen en cinco idiomas diferentes: francés, alemán, español, ruso y turco.

La estructura de los datos es la siguiente:

```
{
    "date": "01/01/2001",
    "summary": "A text",
    "text": "This is a text",
    "title": "A sample",
    "topic": "football",
    "url": "https://www.google.com"
}
```

Para más información: https://huggingface.co/datasets/mlsum

### 2.1. Lectura del conjunto de datos

In [ ]:
from datasets import load_dataset

ds = load_dataset("mlsum", 'es')

In [ ]:
ds

In [ ]:
# Mostramos un ejemplo del subconjunto de datos de entrenamiento
ds["train"]["text"][10]

In [ ]:
# Mostramos el resumen correspondiente al ejemplo anterior
ds["train"]["summary"][10]

### 2.2. Formato del conjunto de datos

Tal y como hemos comentado en secciones anteriores, el conjunto de datos utilizado para aplicar instruction fine-tuning debe estar formado por ejemplos de entrenamiento de la siguiente forma:
```
(prompt, completion)
```
Para formar el prompt debemos tener en cuenta los siguientes puntos:


*   Debe indicarse una instrucción para que realice el modelo. Es habitual utilizar plantillas que proponen los desarrolladores de los LLM para diseñar nuestros ejemplos de entrenamiento: https://github.com/google-research/FLAN/blob/main/flan/v2/flan_templates_branched.py

La plantilla que vamos a seleccionar es la siguiente:


```
("Resume el siguiente articulo:\n\n{text}", "{summary}")
```



*   Para afinar modelos como FLAN-T5 en tareas conversacionales donde queramos preservar el contexto de la conversación, se adopta un enfoque basado en secuencias, donde la interacción entre los participantes de la conversación se estructura en una sola cadena. La pregunta y la respuesta suelen estar separadas por un token especial, como <sep>, <eos>, o simplemente utilizando un delimitador claro (ej: `\n`)



```
("Conversación:\n[Usuario] ¿Cuál es la capital de Francia?\n[Asistente] La capital de Francia es París.\n[Usuario] ¿Y cuál es su río principal?\n[Asistente] ", "Su río principal es el Sena")
```




Reducimos el tamaño del conjunto de datos para consumir menos recursos computacionales

In [ ]:
# Reducimos el conjunto de datos
NUM_EJ_TRAIN = 1500
NUM_EJ_VAL = 500
NUM_EJ_TEST = 200

# Subconjunto de entrenamiento
ds['train'] = ds['train'].select(range(NUM_EJ_TRAIN))

# Subconjunto de validación
ds['validation'] = ds['validation'].select(range(NUM_EJ_VAL))

# Subconjunto de pruebas
ds['test'] = ds['test'].select(range(NUM_EJ_TEST))

In [ ]:
ds

Pre-procesamos el conjunto de datos para aplicar la plantilla seleccionada anteriormente.

In [ ]:
def parse_dataset(ejemplo):
  """Procesa los ejemplos para adaptarlos a la plantilla."""
  return {"prompt": f"Resume el siguiente articulo:\n\n{ejemplo['text']}"}

In [ ]:
ds["train"] = ds["train"].map(parse_dataset)
ds["validation"] = ds["validation"].map(parse_dataset)
ds["test"] = ds["test"].map(parse_dataset)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
ds

In [ ]:
print(ds["train"]["prompt"][10])

In [ ]:
print(ds["train"]["text"][10])

### 2.3. Tokenización del conjunto de datos

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

Una de las cosas que comentabamos cuándo comenzamos a hablar de modelos generativos como la Redes Neuronales Recurrentes, es que este tipo de algoritmos, al igual que los LLMs, reciben secuencias del mismo tamaño.

Con lo cual, al igual que hicimos en ese caso práctico al comienzo del curso, debemos obtener la secuencia más larga de nuestro conjunto de datos y realizar padding al resto de secuencias para que todas tengan el mismo tamaño.

In [ ]:
from datasets import concatenate_datasets

# Calculamos el tamaño máximo de prompt
prompts_tokens = concatenate_datasets([ds["train"], ds["validation"], ds["test"]]).map(lambda x: tokenizer(x["prompt"], truncation=True), batched=True) # Va a truncar en 512 que es el tamaño máximo para este modelo
max_token_len = max([len(x) for x in prompts_tokens["input_ids"]])
print(f"Maximo tamaño de prompt: {max_token_len}")

# Calculamos el tamaño máximo de completion
completions_tokens = concatenate_datasets([ds["train"], ds["validation"], ds["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True)
max_completion_len = max([len(x) for x in completions_tokens["input_ids"]])
print(f"Maximo tamaño de completion: {max_completion_len}")

In [ ]:
def padding_tokenizer(datos):
  # Tokenizar inputs (prompts)
  model_inputs = tokenizer(datos['prompt'], max_length=max_token_len, padding="max_length", truncation=True)

  # Tokenizar labels (completions)
  model_labels = tokenizer(datos['summary'], max_length=max_completion_len, padding="max_length", truncation=True)

  # Sustituimos el caracter de padding de las completion por -100 para que no se tenga en cuenta en el entrenamiento
  model_labels["input_ids"] = [[(l if l != tokenizer.pad_token_id else -100) for l in label] for label in model_labels["input_ids"]]

  model_inputs['labels'] = model_labels["input_ids"]

  return model_inputs

In [ ]:
ds_tokens = ds.map(padding_tokenizer, batched=True, remove_columns=['text', 'summary', 'topic', 'url', 'title', 'date', 'prompt'])

In [ ]:
ds_tokens

In [ ]:
ds_tokens["train"]["input_ids"][10]

## 3. Fine-tuning del modelo

### 3.1. Lectura del modelo

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Cargamos el modelo
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

### 3.2. Evaluación durante el entrenamiento

A continuación implementamos un conjunto de funciones auxiliares para evluar los resultados durante el proceso de entrenamiento

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metrica de evaluación
metric = evaluate.load("rouge")

# Funciona auxiliar para preprocesar el texto
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum espera una nueva línea después de cada frase
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Reemplazamos -100 en las etiquetas porque no podemos decodificarlo
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Preprocesamos el texto
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

### 3.3. Lectura y adaptación de los datos para el entrenamiento

In [ ]:
from transformers import DataCollatorForSeq2Seq

# Ignoramos los tokens relacionados con el padding durante el proceso de entrenamiento para los prompts
label_pad_token_id = -100

# Recolector de datos para el entrenamiento del modelo
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


### Preparación y ejecución del fine-tuning (entrenamiento)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

REPOSITORY="/content/drive/MyDrive/flan-t5-small-fine-tuned"

# Definimos las opciones del entrenamiento
training_args = Seq2SeqTrainingArguments(
    # Hiperprámetros del entrenamiento
    output_dir=REPOSITORY,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=4,
    # Estrategias de logging y evaluación
    logging_dir=f"{REPOSITORY}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Creamos la instancia de entrenamiento
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds_tokens["train"],
    eval_dataset=ds_tokens["validation"],
    compute_metrics=compute_metrics,
)

In [ ]:
# Guardamos el tokenizador en disco para utilizarlo posteriormente
tokenizer.save_pretrained(f"{REPOSITORY}/tokenizer")

In [ ]:
# Iniciamos el entrenamiento
trainer.train()

## 4. Generación de texto con Flan-T5 Fine-tuned y evaluación

### Lectura del modelo y del tokenizador

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

REPOSITORY="/content/drive/MyDrive/flan-t5-small-fine-tuned"

# Importamos el tokenizador
tokenizer_FT5_FT = T5Tokenizer.from_pretrained(f"{REPOSITORY}/tokenizer")

# Importamos el modelo con fine-tuning
model_FT5_FT = T5ForConditionalGeneration.from_pretrained(f"{REPOSITORY}/checkpoint-752", device_map="auto")

### Generación de texto

In [ ]:
text = """Astrónomos detectaron una misteriosa ráfaga de ondas de radio que tardó \
8.000 millones de años en llegar a la Tierra. La ráfaga rápida de radio es una de \
las más distantes y energéticas jamás observadas. Las ráfagas rápidas de radio \
(FRB, por sus siglas en inglés) son intensos estallidos de ondas de radio de \
milisegundos de duración cuyo origen se desconoce. La primera FRB se descubrió \
en 2007 y, desde entonces, se han detectado cientos de estos rápidos destellos \
cósmicos procedentes de puntos distantes de todo el universo."""

In [ ]:
text = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [ ]:
text = """"El telescopio Hubble, lanzado al espacio en 1990, ha proporcionado imágenes \
impresionantes del universo y ha ayudado a los científicos a comprender mejor la cosmología."""

In [ ]:
# Construimos el prompt conforme a la plantilla de fine-tuning
prompt_template = f"Resume el siguiente articulo:\n\n{text}"

# Tokenizamos el prompt
prompt_tokens = tokenizer_FT5_FT(prompt_template, return_tensors="pt").input_ids.to("cuda")

# Generamos los siguientes tokens
outputs = model_FT5_FT.generate(prompt_tokens, max_length=300)

# Transformamos los tokens generados en texto
print(tokenizer_FT5_FT.decode(outputs[0]))

### Evaluación con el subconjunto de pruebas

In [ ]:
import torch

# Cambiamos el modelo al modo de evaluación
model_FT5_FT.eval()

# Definir tamaño del lote
batch_size = 8

all_predictions = []

# Deshabilitamos el entrenamiento y obtenemos las completions
with torch.no_grad():
  for i in range(0, len(ds_tokens["test"]["input_ids"]), batch_size):
        # Extraemos el lote actual
        input_ids_batch = torch.tensor(ds_tokens["test"]["input_ids"][i:i+batch_size], device='cuda:0')

        # Obtenemos las predicciones del modelo
        outputs = model_FT5_FT.generate(input_ids_batch)

        # Concatenemos las predicciones
        all_predictions.extend(outputs)

# Calculamos las metricas
labels = np.array(ds_tokens['test']['labels'])
completions = np.array([pred.cpu().numpy() for pred in all_predictions])

metrics = compute_metrics((completions, labels))

print(metrics)